In [4]:
from dotenv import load_dotenv
import os
import asyncio
import nest_asyncio  # Import nest_asyncio
import httpx  # Import httpx for handling HTTP requests
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import GitbookLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv()

# Access environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize ChatOpenAI with the model
llm = ChatOpenAI(model="gpt-4")

# GitBook URL
gitbook_url = "https://wiki.letsterra.com/"  # Replace with your GitBook URL

def load_gitbook_content():
    # Load GitBook content
    loader = GitbookLoader(gitbook_url, load_all_paths=True)
    all_documents = loader.load()  # No need to await
    return all_documents

async def main():
    # Load documents
    all_documents = load_gitbook_content()  # Call the synchronous function
    print(f"Loaded {len(all_documents)} documents.")  # Debugging line

    # Check if any documents were loaded
    if not all_documents:
        print("No documents were loaded. Please check the GitBook URL or the loading process.")
        return

    # Split the documents into manageable chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=250, add_start_index=True
    )
    all_splits = text_splitter.split_documents(all_documents)

    # Create a new vector store from the documents
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

    # Create a retriever from the vector store
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

    # Define the refined system prompt
    system_prompt = (
        "You are an assistant for question-answering tasks related to game development using a custom game engine called Terra Creator Studio. "
        "Your primary responsibilities include: "
        "- Providing complete function implementations in T# when a script or code snippet is requested. "
        "- Ensure the function code is self-contained and can be directly copied into Terra Studio. "
        "- Prioritize using or modifying existing wrapper functions for logic templates, or combine multiple wrapper functions, rather than writing new functions from scratch unless no suitable options exist. "
        "Follow these guidelines: "
        "- Most syntax is similar to C#, but identify differences by examining access wrappers and methods. "
        "- Refer to the 'T# Don'ts' section of the context document to avoid common pitfalls and differences from Unity C# syntax. "
        "- If a question requests a script, provide complete T# function code that can be copied directly into Terra Studio. "
        "Consistency is key: "
        "- Ensure responses are consistent; similar questions should yield similar answers, even if asked multiple times. "
        "- Always refer to the provided context below. "
        "If you encounter questions about features in Terra Studio: "
        "- Search the relevant portion of the context and provide a relevant answer. "
        "Important: "
        "- Always double-check the context document to ensure your answers are accurate and not based on hallucination. "
        "- Verifying information is more important than providing a quick but incorrect answer. "
        "If you cannot find an answer in the T# documentation, state that you don't know. "
        "Your answers should be: "
        "- Clear, concise, and suitable for novice developers. "
        "- Always include the source of the information used in your response from the context, and ensure the sources are accurate. "
        "- If you cannot provide a source, please indicate that the source is not available."
        "\n\n"
        "{context}\n\n"
        "Sources:\n{sources}"
    )

    # Create the chat prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    # Create the question-answer chain
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    # Store for chat history
    store = {}
    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]

    # Wrap the RAG chain with message history management
    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )

    session_id = "ashw0014"  # Unique session ID for the conversation

    # Test the retrieval-augmented generation chain
    input_question = "How do I access haptics via T# code"
    try:
        retrieved_docs = retriever.invoke(input_question)

        # Check if any documents were retrieved
        if not retrieved_docs:
            print("No documents retrieved. Please check your query.")
        else:
            context = "\n\n".join(doc.page_content for doc in retrieved_docs)

            # Create a more informative sources output
            sources = "\n".join(f"- Source: {doc.metadata['source']}\n  Content: {doc.page_content[:300]}..." for doc in retrieved_docs)  # Show first 300 characters

            # Retrieve the chat history for the session
            chat_history = get_session_history(session_id)

            # Include chat history in the input
            input_with_history = "\n".join([f"User: {msg['input']}\nAI: {msg['answer']}" for msg in chat_history.messages]) + f"\nUser: {input_question}"

            formatted_prompt = system_prompt.format(context=context, sources=sources)

            response = conversational_rag_chain.invoke(
                {"input": input_question, "context": context, "sources": sources},
                config={"configurable": {"session_id": session_id}}
            )

            # Store the new question and answer in the chat history
            chat_history.messages.append({"input": input_question, "answer": response["answer"]})

            print(response["answer"])

    except httpx.ConnectError as e:
        print(f"Connection error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the main function
if __name__ == "__main__":
    asyncio.run(main())

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Fetching pages: 100%|##########| 91/91 [00:12<00:00,  7.26it/s]


Loaded 91 documents.
In T#, you can access haptics via the StudioHaptics class. This class provides several static methods to play different haptic feedback patterns. Here are some examples:

1. To play the haptic feedback for selection:
```T#
StudioHaptics.PlayHapticSelection();
```

2. To play the haptic feedback for success:
```T#
StudioHaptics.PlayHapticSuccess();
```

3. To play the haptic feedback for warning:
```T#
StudioHaptics.PlayHapticWarning();
```

These methods will trigger haptic feedback on supported devices.

Source: [T# Haptics & Extensions](https://wiki.letsterra.com/coding-using-t/t-haptics-and-extensions)
